In the poker week, PPO was broken - reward/signal was flowing between episodes. The purpose of this notebook is to fix this and maybe parallelize it better. 

In [2]:
import random
import math
from copy import deepcopy

from tqdm import tqdm_notebook
import numpy as np
import torch
from torch import nn
from torch.distributions import Categorical
from collections import deque
from typing import Tuple
from einops import rearrange, repeat

# from check_submission import check_submission
from game_mechanics import *
from functools import partial

TEAM_NAME = "Bulgar Stronk"

%load_ext autoreload
%autoreload 2

In [9]:
# Hyperparameters
gamma = 0.99
lamda = 0.95
epsilon = 0.2
beta = 0.01

batch_size = 1024
num_episodes = 100_000
reset_opponent_gap = 3_000
epochs_per_batch = 1

pol_lr = 0.001
n_pol_neurons = 256
val_lr = 0.01
n_val_neurons = 32

# Policy net
policy = nn.Sequential(
    nn.Linear(10, 2),
    nn.Softmax()
)

pol_optim = torch.optim.Adam(policy.parameters(), lr=pol_lr)

# Value net
V = nn.Sequential(
    nn.Linear(10, 1),
)

val_optim = torch.optim.Adam(V.parameters(), lr=val_lr)
val_loss_fn = nn.MSELoss()

In [11]:
initial_state = torch.zeros(10)

In [34]:
torch.rand(10)

tensor([0.0714, 0.9216, 0.4027, 0.2005, 0.0934, 0.9643, 0.9513, 0.2475, 0.4475,
        0.7969])

In [49]:
def play_game():
    state = torch.zeros(10)
    states = []
    successor_states = []
    moves = []
    rewards = []
    values = []
    done = False
    while not done:
        move = torch.argmax(policy(state)).item()
        if random.random() > 0.9:
            done = True
        new_state = torch.rand(10)
        
        states.append(state)
        successor_states.append(new_state)
        moves.append(move)
        rewards.append(done*10)
        state = new_state
    return states, successor_states, moves, rewards

In [58]:
s, ss, m, r = play_game()
print(len(s))

3


In [60]:
s

[tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 tensor([0.8080, 0.7543, 0.7743, 0.3747, 0.3730, 0.8463, 0.3515, 0.2142, 0.4241,
         0.5462]),
 tensor([0.7316, 0.1167, 0.7791, 0.1339, 0.0445, 0.9479, 0.6434, 0.4083, 0.4323,
         0.6407])]

In [59]:
ss

[tensor([0.8080, 0.7543, 0.7743, 0.3747, 0.3730, 0.8463, 0.3515, 0.2142, 0.4241,
         0.5462]),
 tensor([0.7316, 0.1167, 0.7791, 0.1339, 0.0445, 0.9479, 0.6434, 0.4083, 0.4323,
         0.6407]),
 tensor([0.9006, 0.9039, 0.0562, 0.6037, 0.2010, 0.7868, 0.4241, 0.2797, 0.8509,
         0.1175])]